In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import networkx
import Graph

g = Graph.buildGraph()
# mulGraph = Graph.buildGraph()

Connection pairs:
[
    (7529, 3491, 489.50118772441493),
    (7483, 6868, 128.34041442898507),
    (7487, 7090, 100.0674332727893),
    (7492, 6907, 44.872694234504756),
    (7497, 6861, 45.77161729321111),
    (7498, 6863, 15.894429251450745),
    (7504, 672, 71.5994991372535),
    (7511, 234, 11.285070412497834),
    (7516, 4166, 19.355601312460404),
    (7685, 7508, 26.251222182548965),
    (7487, 7485, 34.69950831729087),
    (7498, 7483, 133.01878867990203),
    (7499, 7489, 161.861689287105),
    (7683, 7518, 839.9571389347612)
]

7529 3491 time = 305.9382423277593 dis = 489.50118772441493 routeVar = Walk

7483 6868 time = 80.21275901811566 dis = 128.34041442898507 routeVar = Walk

7487 7090 time = 62.54214579549331 dis = 100.0674332727893 routeVar = Walk

7492 6907 time = 28.04543389656547 dis = 44.872694234504756 routeVar = Walk

...

7487 7485 time = 21.68719269830679 dis = 34.69950831729087 routeVar = Walk

7498 7483 time = 83.13674292493876 dis = 133.01878867990203 routeVar = Walk

7499 7489 time = 101.16355580444062 dis = 161.861689287105 routeVar = Walk

7683 7518 time = 524.9732118342257 dis = 839.9571389347612 routeVar = Walk

Number of subgraphs: 1

## Dijkstra Algorithm

### Print out path for Dijkstra

In [14]:
def traversePath(paths: dict, g: networkx.MultiDiGraph, u, v):
    if v not in paths:
        # print(f"{v}{(v not in paths)=}")
        return [], -1
    
    if u not in paths:
        # print(f"{u}{(u not in paths)=}")
        return [], -1
    
    path = []
    weight = 0
    while u != v:
        weight += g[paths[v][0]][v][0]['time']
        inbetweenPath = g[paths[v][0]][v][0].get('path', [])
        if inbetweenPath == []:
            path.insert(0, v)
        else:
            path.insert(0, inbetweenPath)
        v = paths[v][0]
        
    path.insert(0, u)
    return path, weight


In [19]:
# Correct Dijkstra
import heapq
import rich 

def Dijkstra1(g: networkx.MultiDiGraph, index = 0):
    minHeap = [[0, list(g.nodes)[index], list(g.nodes)[index]]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    for i in list(g.nodes):
        if i not in shortestPath:
            shortestPath[i] = (-1, -1) 
            
    return shortestPath

shortest = Dijkstra1(g, 0)
path, totalT = traversePath(shortest, g, 35, 7265)
print("Path:", path, 'in', totalT)

Path:
[35, 1451, 27, 7265]
in 4.5538324026023265

## Hierachy Contraction

In [16]:
import heapq
def DijkstraFromTo(g: networkx.MultiDiGraph, start, end, maxLimit = -1):
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        if maxLimit != -1:
            if maxLimit < weight:
                return shortestPath, shortestPath.get(end, -1)
        if cur == end:
            break
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    for i in list(g.nodes):
        if i not in shortestPath:
            shortestPath[i] = (-1, -1) 
            
    return shortestPath, shortestPath[end][1]

shortest, w = DijkstraFromTo(g, 35, 7265)
path, totalT = traversePath(shortest, g, 35, 7265)
print("Path:", path, 'in', totalT)

Path: [35, 1451, 27, 7265] in 4.5538324026023265


In [4]:
import heapq
import rich
import time

def dijkstraLocalSearch(g: networkx.MultiDiGraph, startingNode, endings: list, maxTotal = 30):
    start = time.time()
    # g = g.copy()
    # g.remove_node(avoid)
    endingNodes = endings.copy()
    minHeap = [[0, startingNode, startingNode, 0]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro, hop = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        if cur in endingNodes:
            endingNodes.remove(cur)
        
        if hop > 4:
            continue
        
        # if len(endingNodes) == 0 or weight > maxTotal or len(shortestPath) > 10:
        if len(endingNodes) == 0 or weight > maxTotal:
            # if len(shortestPath) > 10:
            #     print('Too much in shortestPath')
            break
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur, hop + 1])
    
    # for i in list(g.nodes):
    #     if i not in shortestPath:
    #         shortestPath[i] = (-1, -1) 
            
    # pureShortestPath = {}
    # for i in endings:
    #     if i in shortestPath:
    #         pureShortestPath[i] = shortestPath[i]
    if time.time() - start > .2:
        print('Its too long:', time.time() - start)
    return shortestPath
    
def findShortcuts(g: networkx.MultiDiGraph, x, debug = False):
    pre = list(g.predecessors(x))
    succ = list(g.successors(x))
    shorts = []
    pth = {}
    
    if debug:
        print(f"{pre=}")
        print(f"{succ=}")
    
    pMax = {}
    for u in pre:
        for v in succ:
            pMax[u] = max(g[u][x][0]['time'] + g[x][v][0]['time'], pMax.get(u, 0))
            
    # print(succ)
    # print('Pre:', pre)
    # rich.print(pMax)
    
    for u in pre:
        pth = dijkstraLocalSearch(g, u, succ, pMax[u])
        # rich.print(pMax[u], pth)
        # rich.print()
        for v in succ:
            if networkx.is_path(g, [u, v]):
                if debug:
                    print('Already path:', f"{u}->{v}")
                continue
            # shortestDict, weight = DijkstraFromTo(g, u, v)
            path, weight = traversePath(pth, g, u, v)
            shortcutWeight = g[u][x][0]['time'] + g[x][v][0]['time']
            if debug:
                print('-------')
                print(f"{u=}, {v=}", end=':')
                print(f"{(weight == shortcutWeight)=}")
            # if weight == -1:
            #     continue
            if (weight == shortcutWeight) or weight == -1:
            # if weight == shortcutWeight:
                shorts.append((u, v, shortcutWeight))
            # break
        # break
    
    return shorts

findShortcuts(g, 27)

[(26, 7266, 3.223188411343297),
 (26, 30, 3.138033704260138),
 (26, 7264, 3.1102615811811516),
 (26, 7267, 3.3901319297800367),
 (26, 7265, 3.151462946416166),
 (26, 28, 2.1143908899898194),
 (26, 4275, 3.4688396389122658),
 (1379, 7266, 3.5436997675377504),
 (1379, 30, 3.458545060454591),
 (1379, 7264, 3.430772937375605),
 (1379, 7267, 3.71064328597449),
 (1379, 7265, 3.471974302610619),
 (1379, 28, 2.4349022461842726),
 (1379, 4275, 3.789350995106719),
 (1451, 7266, 3.079648060605866),
 (1451, 30, 2.9944933535227065),
 (1451, 7264, 2.96672123044372),
 (1451, 7267, 3.2465915790426054),
 (1451, 7265, 3.0079225956787345),
 (1451, 28, 1.9708505392523878),
 (1451, 4275, 3.3252992881748344),
 (7276, 30, 4.139487295285775),
 (7276, 7264, 4.111715172206789),
 (7276, 7267, 4.391585520805673),
 (7276, 28, 3.1158444810154564)]

In [35]:
print(g)

MultiDiGraph with 4397 nodes and 9974 edges


In [6]:
# Benchmark
start = time.time()
allNodes = list(g.nodes)
minHeap = []
for node in allNodes:
    heapq.heappush(minHeap, [edgeDiff(g, node), node])
print('Init ED:', time.time() - start)

Init ED: 0.11682462692260742


In [72]:
# Testing short intergrity
count = 0
for short in findShortcuts(g, 27, 1):
    if networkx.is_path(g, [short[0], short[1]]):
        count += 1
        
print('Wrong shorts:', count)

pre=[26, 1379, 1451, 7276]
succ=[7266, 30, 7264, 7267, 7265, 28, 4275]
-------
u=26, v=7266:(weight == shortcutWeight)=True
-------
u=26, v=30:(weight == shortcutWeight)=True
-------
u=26, v=7264:(weight == shortcutWeight)=True
-------
u=26, v=7267:(weight == shortcutWeight)=True
-------
u=26, v=7265:(weight == shortcutWeight)=True
-------
u=26, v=28:(weight == shortcutWeight)=True
-------
u=26, v=4275:(weight == shortcutWeight)=True
-------
u=1379, v=7266:(weight == shortcutWeight)=True
-------
u=1379, v=30:(weight == shortcutWeight)=True
-------
u=1379, v=7264:(weight == shortcutWeight)=True
-------
u=1379, v=7267:(weight == shortcutWeight)=True
-------
u=1379, v=7265:(weight == shortcutWeight)=True
-------
u=1379, v=28:(weight == shortcutWeight)=True
-------
u=1379, v=4275:(weight == shortcutWeight)=True
-------
u=1451, v=7266:(weight == shortcutWeight)=True
-------
u=1451, v=30:(weight == shortcutWeight)=True
-------
u=1451, v=7264:(weight == shortcutWeight)=True
-------
u=1451, v=

In [65]:
networkx.is_path(g, [1396, 1451])
networkx.dijkstra_path(g, 1396, 1451)

[1396, 35, 1451]

In [5]:
def edgeDiff(g: networkx.MultiDiGraph, x):
    incomingNode = len(list(g.predecessors(x)))
    outgoingNode = len(list(g.successors(x)))
    shorts = len(findShortcuts(g, x))
    return shorts - (incomingNode + outgoingNode)

edgeDiff(g, 35)

10

In [7]:
import heapq


def Contract(g: networkx.MultiDiGraph, x, order):
    start = time.time()
    shortcuts = findShortcuts(g, x)
    # print(x, shortcuts)
    for short in shortcuts:
        incomingEdge = g.get_edge_data(short[0], x)[0]
        outgoingEdge = g.get_edge_data(x, short[1])[0]
        contractedNodes = incomingEdge.get('path', []) + [x] + outgoingEdge.get('path', [])
        # rich.print(x, short, incomingEdge.get('path', []), outgoingEdge.get('path', []))
        # return
        # print(short[0], short[1], short[2], incomingEdge['dis'] + outgoingEdge['dis'], contractedNodes)
        g.add_edge(short[0], short[1], time = short[2], dis = incomingEdge['dis'] + outgoingEdge['dis'], path = contractedNodes)

    if time.time() - start > 1:
        print('Its too long:', time.time() - start, 'at', x)
        
def AddToAllIncidentNode(g: networkx.MultiDiGraph, node, minHeap: list):
    pred = list(g.predecessors(node))
    succ = list(g.successors(node))
    
    incident = pred + succ
    
    for i in minHeap:
        if i[1] in incident:
            minHeap.remove(i)
            heapq.heappush(minHeap, [i[0] + 1, i[1]])
        
remaining = []
savingG = 1
def PreprocessingPhrase(d: networkx.MultiDiGraph):
    g = d.copy()
    
    allNodes = list(g.nodes)
    
    minHeap = []
    for node in allNodes:
        heapq.heappush(minHeap, [edgeDiff(g, node), node])
        
    order = 1
    print('Min:', minHeap[0])
    ma = 0
    ogMinSize = len(minHeap)
    print('Og size:', ogMinSize)
    startMin = time.time()
    timer= {}
    while len(minHeap) > 0:
        start = time.time()
        
        node = minHeap[0]
        # diff = edgeDiff(g, node[1])
        endDiff = time.time()
        timer['Recalc diff'] = endDiff - start
        
        # if (diff > node[0]):
        if (False):
            heapq.heappop(minHeap)
            heapq.heappush(minHeap, [diff, node[1]])
        else:
            Contract(g, node[1], order)
            ma = node[1]
            # Contract(g, 35, order)
            networkx.set_node_attributes(g, {node[1]: {'order': order}})
            order += 1
            heapq.heappop(minHeap)
            AddToAllIncidentNode(g, node[1], minHeap)
        # return
        timer['Contrt nodes'] = time.time() - endDiff
        
        if time.time() - start > 2:
            print('Too long')
            print(timer)
        
        savingG = g
        if time.time() - startMin > 3:
            print(timer)
            print('Process:', (len(minHeap)), 'left, in', time.time() - startMin, 'at', ma)
            remaining = minHeap
            startMin = time.time()
    # rich.print(minHeap)
    print('Max:', ma)
    return g
    
contracted = PreprocessingPhrase(g)
savingG

Min: [-6, 7265]
Og size: 4397
{'Recalc diff': 0.0, 'Contrt nodes': 0.0028676986694335938}
Process: 353 left, in 3.002018451690674 at 1631
{'Recalc diff': 0.0, 'Contrt nodes': 0.1758279800415039}
Process: 192 left, in 3.05416202545166 at 759
{'Recalc diff': 0.0, 'Contrt nodes': 0.5248510837554932}
Process: 151 left, in 3.336242437362671 at 1
{'Recalc diff': 0.0, 'Contrt nodes': 0.2127094268798828}
Process: 127 left, in 3.1269657611846924 at 1003
{'Recalc diff': 0.0, 'Contrt nodes': 0.4899885654449463}
Process: 110 left, in 3.1458327770233154 at 25
{'Recalc diff': 0.0, 'Contrt nodes': 0.4593055248260498}
Process: 96 left, in 3.323629140853882 at 1508
{'Recalc diff': 0.0, 'Contrt nodes': 0.5340323448181152}
Process: 87 left, in 3.3146469593048096 at 281
Its too long: 0.20397377014160156
Its too long: 1.2797486782073975 at 4166
{'Recalc diff': 0.0, 'Contrt nodes': 1.2797486782073975}
Process: 82 left, in 3.4024853706359863 at 4166
{'Recalc diff': 0.0, 'Contrt nodes': 0.4755589962005615}
Pr

1

In [9]:
print(g)
print(contracted)

MultiDiGraph with 4397 nodes and 9974 edges
MultiDiGraph with 4397 nodes and 678071 edges


In [43]:
print(contracted)

MultiDiGraph with 4397 nodes and 919885 edges


In [276]:
cop = g.copy()
print(networkx.is_path(cop, [7270, 86]))
rich.print(findShortcuts(cop, 35))
Contract(cop, 35, 4)

print()
print(networkx.is_path(cop, [7270, 86]))
print(networkx.is_path(cop, [7275, 86]))

print()
Contract(cop, 35, 4)
findShortcuts(cop, 35)

False


[
    (7270, 86, 5.908830966850482),
    (7270, 89, 6.327860087159596),
    (7275, 89, 2.95671797063625),
    (377, 1451, 9.624780862141622),
    (377, 7265, 14.9224646610568),
    (1396, 89, 4.640970199721236)
]


True
True



[]

In [12]:
def QueryPhrase(g: networkx.MultiDiGraph, start):
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        # if cur == end:
        #     break
        
        for u in g.succ[cur]:
            # print(g.nodes[u]['order'], g.nodes[start]['order'])
            if u not in shortestPath and g.nodes[u]['order'] > g.nodes[cur]['order']:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    # print(shortestPath)
    return shortestPath

def QueryPhraseReversed(g: networkx.MultiDiGraph, start):
    print('Waiting for reversing graph...')
    g = g.reverse()
    print('Done reversing')
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        # if cur == end:
        #     break
        
        for u in g.succ[cur]:
            # print(g.nodes[u]['order'], g.nodes[start]['order'])
            if u not in shortestPath and g.nodes[u]['order'] > g.nodes[cur]['order']:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    # print(shortestPath)
    return shortestPath
    
fromWhere = 35
toWhere = 501
d1 = QueryPhrase(contracted, fromWhere)
print('Forward query done')
d2 = QueryPhraseReversed(contracted, toWhere)
print('Backward query done')

final_dict = {x:d1[x] for x in d1 if x in d2}
# for x in d1:
#     print(x)
#     if x in d2:
#         final_dict[x] = 1
#         print('YES')
#     else:
#         print('Not in d2')
# print(35 in d2)

# rich.print(len(final_dict))
# return

minSoFar = -1
middle = -1
# print(len(d1))
# rich.print((d2))
for key in final_dict:
    print('Pair exist on both dictionaries:', d1[key][0] , d2[key][0], d1[key][1] + d2[key][1])
    if minSoFar == -1 or minSoFar > d1[key][1] + d2[key][1]:
        minSoFar = d1[key][1] + d2[key][1]
        middle = key
        
print('From:', fromWhere, 'to', toWhere)
print('From Hierarchy contraction', minSoFar, 'through the stop', middle)
# print(networkx.dijkstra_path_length(contracted, fromWhere, toWhere, 'time'))
idc, result = DijkstraFromTo(g, fromWhere, toWhere)
print('From Djikstra algorithm', result, traversePath(idc, g, fromWhere, toWhere))

Forward query done
Waiting for reversing graph...
Done reversing
Backward query done
Pair exist on both dictionaries: 35 89 23.700418229778258
Pair exist on both dictionaries: 35 501 23.700418229778258
From: 35 to 501
From Hierarchy contraction 23.700418229778258 through the stop 35
From Djikstra algorithm 23.70041822977826 ([35, 89, 385, 387, 3182, 433, 434, 436, 397, 137, 617, 618, 619, 496, 502, 499, 542, 501], 23.700418229778258)


In [304]:
# rich.print(findShortcuts(g, 27))
# contracted[35][7276][0]
d1[7277]

(7276, 1.8328662072033386)

In [46]:
print(g[27][7265][0])
print(g[1451][27][0])
print('Check if theres a path:', networkx.is_path(g, [1451, 7265]))
sc = findShortcuts(g, 27, True)
sc


{'time': 2.4030455066844705, 'dis': 594.2855502937753, 'routeVar': ('66', '1')}
{'time': 0.6048770889942638, 'dis': 145.62968837576037, 'routeVar': ('59', '118')}
Check if theres a path: False
pre=[26, 1379, 1451, 7276]
succ=[7266, 30, 7264, 7267, 7265, 28, 4275]
-------
u=26, v=7266:(weight == shortcutWeight)=True
-------
u=26, v=30:(weight == shortcutWeight)=True
-------
u=26, v=7264:(weight == shortcutWeight)=True
-------
u=26, v=7267:(weight == shortcutWeight)=True
-------
u=26, v=7265:(weight == shortcutWeight)=True
-------
u=26, v=28:(weight == shortcutWeight)=True
-------
u=26, v=4275:(weight == shortcutWeight)=True
-------
u=1379, v=7266:(weight == shortcutWeight)=True
-------
u=1379, v=30:(weight == shortcutWeight)=True
-------
u=1379, v=7264:(weight == shortcutWeight)=True
-------
u=1379, v=7267:(weight == shortcutWeight)=True
-------
u=1379, v=7265:(weight == shortcutWeight)=True
-------
u=1379, v=28:(weight == shortcutWeight)=True
-------
u=1379, v=4275:(weight == shortcutW

[(26, 7266, 3.223188411343297),
 (26, 30, 3.138033704260138),
 (26, 7264, 3.1102615811811516),
 (26, 7267, 3.3901319297800367),
 (26, 7265, 3.151462946416166),
 (26, 28, 2.1143908899898194),
 (26, 4275, 3.4688396389122658),
 (1379, 7266, 3.5436997675377504),
 (1379, 30, 3.458545060454591),
 (1379, 7264, 3.430772937375605),
 (1379, 7267, 3.71064328597449),
 (1379, 7265, 3.471974302610619),
 (1379, 28, 2.4349022461842726),
 (1379, 4275, 3.789350995106719),
 (1451, 7266, 3.079648060605866),
 (1451, 30, 2.9944933535227065),
 (1451, 7264, 2.96672123044372),
 (1451, 7267, 3.2465915790426054),
 (1451, 7265, 3.0079225956787345),
 (1451, 28, 1.9708505392523878),
 (1451, 4275, 3.3252992881748344),
 (7276, 30, 4.139487295285775),
 (7276, 7264, 4.111715172206789),
 (7276, 7267, 4.391585520805673),
 (7276, 28, 3.1158444810154564)]

In [ ]:
# print('Check if theres a path:', networkx.is_path(contracted, [1451, 7265]))
# print(1451 in g.predecessors(27))
shorts = findShortcuts(g, 27, True)
rich.print('Shortcuts to make:', (shorts))
count = 0
for short in shorts:
    if networkx.is_path(g, [short[0], short[1]]):
        count += 1
        
print('Wrong shorts:', count)
print('Check if theres a path:', networkx.is_path(g, [1451, 7265]))
# print(networkx.is_path(contracted, [1451, 7265]))
# print('SC if no middle node', traversePath(dijkstraLocalSearch(g, 1451, 27, [7265]), g, 1451, 7265))

# traversePath(DijkstraFromTo(g, 1451, 7265)[0], g, 1451, 7265)

In [268]:
# for i in networkx.predecessor(contracted, 27): print(i)
# len(list(networkx.predecessor(contracted, 27)))
# print(findShortcuts(contracted, 27))
# contracted.pr
print(findShortcuts(contracted, 27))
for i in contracted.successors(27):
    # print(i, 'order=', contracted.nodes[i]['order'])
    # print(networkx.is_path(contracted, [i, 27]))
    for u in contracted.successors(27):
        # print(u)
        if networkx.is_path(contracted, [i, u]):
            print(i, u, contracted[i][u])
# contracted[7265][1451]
# testG = 


[(7276, 7264, 4.111715172206789)]
7265 7266 {0: {'time': 0.1970180148729217, 'dis': 60.10733528493736, 'routeVar': ('3', '5')}}


In [ ]:
G = networkx.MultiDiGraph()
D = networkx.path_graph(4)
G.add_edges_from([(1, 2), (2, 3), (3, 4)])
print(networkx.is_path(G, [2, 3]))
print(list(G.predecessors(3)))
# print(list(G.pred[3]))
# print(list(G.predecessors(3)))
# networkx.is_path(G, [1, 3])
D = networkx.path_graph(4)
# networkx.is_path(G, [2, 1])
# list(G.predecessors(4))


True
[2]


In [ ]:
# findShortcuts(contracted, 35)
dc = g.copy()
print(dc)
Contract(dc, 35, 4)
Contract(dc, 35, 4)
print(dc)
findShortcuts(dc, 35)
# networkx.is_path(7275, 86)

MultiDiGraph with 4397 nodes and 9974 edges
MultiDiGraph with 4397 nodes and 9982 edges


[(7275, 86, 2.5376888503271355)]

In [ ]:
import heapq

def DijkstraFrom(g: networkx.MultiDiGraph, start, allOutgoingNodes: dict):
    allOutgoingNodes = allOutgoingNodes.copy()
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        if cur in allOutgoingNodes:
            allOutgoingNodes.pop(cur)
            
        if len(allOutgoingNodes) == 0:
            break
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    for i in g.nodes:
        if i not in shortestPath:
            shortestPath[i] = (-1, -1) 
    
    return shortestPath
    
DijkstraFrom(g, 7526, [])

{7526: (7526, 0),
 35: (-1, -1),
 7276: (-1, -1),
 7277: (-1, -1),
 7278: (-1, -1),
 7265: (-1, -1),
 7266: (-1, -1),
 7693: (-1, -1),
 1256: (-1, -1),
 7588: (-1, -1),
 32: (-1, -1),
 31: (-1, -1),
 34: (-1, -1),
 42: (-1, -1),
 44: (-1, -1),
 39: (-1, -1),
 41: (-1, -1),
 43: (-1, -1),
 46: (-1, -1),
 47: (-1, -1),
 45: (-1, -1),
 49: (-1, -1),
 48: (-1, -1),
 50: (-1, -1),
 51: (-1, -1),
 52: (-1, -1),
 53: (-1, -1),
 54: (-1, -1),
 55: (-1, -1),
 56: (-1, -1),
 57: (-1, -1),
 60: (-1, -1),
 58: (-1, -1),
 62: (-1, -1),
 67: (-1, -1),
 63: (-1, -1),
 72: (-1, -1),
 64: (-1, -1),
 4754: (-1, -1),
 73: (-1, -1),
 213: (-1, -1),
 68: (-1, -1),
 74: (-1, -1),
 69: (-1, -1),
 70: (-1, -1),
 71: (-1, -1),
 77: (-1, -1),
 3204: (-1, -1),
 3203: (-1, -1),
 3206: (-1, -1),
 3259: (-1, -1),
 3207: (-1, -1),
 3210: (-1, -1),
 75: (-1, -1),
 3255: (-1, -1),
 3208: (-1, -1),
 3257: (-1, -1),
 3205: (-1, -1),
 3261: (-1, -1),
 3260: (-1, -1),
 3262: (-1, -1),
 3263: (-1, -1),
 204: (-1, -1),
 205

In [ ]:
print(len(list(networkx.neighbors(g, 1197))))

2

In [ ]:
from rich.progress import track
import time
import json

def nodeNet(g: networkx.MultiDiGraph, node):
    firstTime = True
    shortcuts = 0
    shortcutPairs = []
    pre = g.pred[node]
    succ = g.succ[node]
    for u in pre:
        start = time.time()
        shortestPath = DijkstraFrom(g, u, succ)
        if time.time() - start > .5:
            if firstTime:
                print("----------------")
                firstTime = False
            print(f"{len(pre) = }")
            print(f"{len(succ) = }")
            print(u, time.time() - start)
            
        for w in succ:
            if networkx.is_path(g, [u, w]):
                continue
            totalTime = pre[u][0]['time'] + succ[w][0]['time']
            if totalTime == shortestPath[w][1]:
                shortcuts += 1
                shortcutPairs.append((u, w))
                
    # print(f"{shortcuts = }, {len(pre) = }, {len(succ) = }")
    return shortcuts - (len(pre) + len(succ)), shortcutPairs

def contract(g: networkx.MultiDiGraph, x, contractionOrdder):
    start = time.time()
    stuff, edges = nodeNet(g, x)
    # print(edges)
    if len(edges) > 1000:
        print(len(edges))
    if (time.time() - start > 1):
        print("Node Net:", time.time() - start)
    startE = time.time()
    for edge in edges:
        # print(g.get_edge_data(edge[0], x))
        # if (edge[0] == 35 and edge[1] == 7483):
        #     print("Found it")
        uPath = g.get_edge_data(edge[0], x)[0].get('path', [])
        uData = g.get_edge_data(edge[0], x)[0]
        wPath = g.get_edge_data(x, edge[1])[0].get('path', [])
        wData = g.get_edge_data(x, edge[1])[0]
        # print(f"{uPath = }, {wPath = }")
        # print(f"{uData = }, {wData = }")
        g.add_edge(edge[0], edge[1], time = uData['time'] + wData['time'], dis = uData['dis'] + wData['dis'], path = uData.get('path', []) + [x] + wData.get('path', []))
    if time.time() - startE > 1:
        print("Edges:", time.time() - startE)

def contractionPhrase(d: networkx.MultiDiGraph):
    g: networkx.MultiDiGraph = d.copy()
    edgeDifference = {}
    nodeList = list(g.nodes)
    for x in nodeList:
        edgeDifference[x] = nodeNet(g, x)
        
    contractionOrder = 1
    for x in ((nodeList)):
        # print("Contracting", x, end='')
        x = min(edgeDifference, key=edgeDifference.get)
        contractedIncidentNodes = 0
        start = time.time()
        timed = {}
        for pre in g.pred[x]:
            if edgeDifference.get(pre, None) is None:
                contractedIncidentNodes += 1
                continue
            newTuple = (edgeDifference[pre][0] + 1, edgeDifference[pre][1])
            # print(newTuple)
            edgeDifference.pop(pre)
            edgeDifference[pre] = newTuple
        timed['pre'] = time.time() - start
        startPre = time.time()
        # print("pre")
        for succ in g.succ[x]:
            if edgeDifference.get(succ, None) is None: 
                contractedIncidentNodes += 1
                continue
            newTuple = (edgeDifference[succ][0] + 1, edgeDifference[succ][1])
            edgeDifference.pop(succ)
            edgeDifference[succ] = newTuple
        timed['succ'] = time.time() - startPre
        startSucc = time.time()
        # print('succ')
        # if contractedIncidentNodes < 10:
        contract(g, x, contractionOrder)
        timed['con'] = time.time() - startSucc
        
        networkx.set_node_attributes(g, {x: {"order": contractionOrder}})
        contractionOrder += 1
        edgeDifference.pop(x)
        # print("Node", x, len(list(g.neighbors(x))))
        if (time.time() - start > 5):
            print("Finished", x, "in", time.time() - start, timed)
        # print("remove", x)
    
    print(g)
    return g
    
ed = contractionPhrase(g)

Node Net: 1.0555016994476318
Node Net: 1.6599907875061035
Node Net: 1.347472906112671
Node Net: 1.2241151332855225
Node Net: 1.0483098030090332
Node Net: 2.707213878631592
Node Net: 1.525414228439331
Node Net: 1.999441385269165
Node Net: 1.7928552627563477
Node Net: 1.6543991565704346
Node Net: 2.837576150894165
1017
Node Net: 1.0612475872039795
Node Net: 1.443526029586792
Node Net: 3.800062894821167
Node Net: 2.404812812805176
2146
Node Net: 3.9762635231018066
Node Net: 1.4820826053619385
1564
Node Net: 1.178849697113037
1382
Node Net: 2.270646095275879
1050
Node Net: 2.4456465244293213
1223
Node Net: 1.6479003429412842
Node Net: 2.233660936355591
1278
Node Net: 4.887126684188843
1408
4184
Node Net: 7.453658580780029
Finished 725 in 7.466386079788208 {'pre': 0.0, 'succ': 0.0, 'con': 7.466386079788208}
2912
Node Net: 12.60015606880188
Finished 1920 in 12.60715627670288 {'pre': 0.0, 'succ': 0.0, 'con': 12.60715627670288}
Node Net: 6.954506158828735
Finished 347 in 6.95750617980957 {'pre

In [ ]:
print(g)

MultiDiGraph with 4397 nodes and 9974 edges


In [ ]:
test = ed.copy()
test.nodes[35]

{'coord': (684720.4742496928, 1190800.3385383594), 'order': 4395}

In [ ]:
DijkstraFromTo(ed, 35, 27)

2.150786895917856

In [ ]:
def queryPhrase(g: networkx.MultiDiGraph, start):
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        # if cur == end:
        #     break
        
        for u in g.succ[cur]:
            # print(g.nodes[u]['order'], g.nodes[start]['order'])
            if u not in shortestPath and g.nodes[u]['order'] > g.nodes[start]['order']:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    # print(shortestPath)
    return shortestPath
    
def queryPhraseReversed(g: networkx.MultiDiGraph, start):
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        # if cur == end:
        #     break
        
        for u in g.pred[cur]:
            # print(g.nodes[u]['order'], g.nodes[start]['order'])
            if u not in shortestPath and g.nodes[u]['order'] > g.nodes[start]['order']:
                heapq.heappush(minHeap, [weight + g[u][cur][0]['time'], u, cur])
    
    # print(shortestPath)
    return shortestPath
    
fromWhere = 35
toWhere = 7483
d1 = queryPhrase(ed, fromWhere)
d2 = queryPhraseReversed(ed, toWhere)

final_dict = {x:d1[x] for x in d1 if x in d2}

# print(d1)
# print(d2)
# print(final_dict)

minSoFar = -1
middle = -1
for key in final_dict:
    if minSoFar == -1 or minSoFar > d1[key][1] + d2[key][1]:
        minSoFar = d1[key][1] + d2[key][1]
        middle = key
        
print(minSoFar, key)
print(DijkstraFromTo(g, fromWhere, toWhere))
# print(ed.nodes[27]['order'])
for i in networkx.dijkstra_path(g, fromWhere, toWhere):
    print(i, ":", ed.nodes[i]['order'])
print(networkx.dijkstra_path(g, fromWhere, toWhere))
# print(networkx.dijkstra_path_length(g, 14, 27, 'time'))
# print(networkx.dijkstra_path_length(ed, 14, 27, 'time'))

NameError: name 'ed' is not defined

In [ ]:
d1

{35: (35, 0)}

In [ ]:
while len(hepa) > 0:
    if (hepa[0][1] == 35 or hepa[0][1] == 7483):
        print("Here")
    print(heapq.heappop(hepa))

[1, 7265]

[196, 3683]

[238, 2114]

[587, 2122]

[593, 857]

...

[4338, 859]

[4386, 1256]

[4392, 27]

Here

[4395, 35]

In [ ]:
print(min(ed, key=ed.get))
ed[min(ed, key=ed.get)][0] += 1

7265

TypeError: 'tuple' object does not support item assignment

In [ ]:
di = networkx.floyd_warshall(g, 'time')

In [ ]:
from rich.progress import track
import time
import time

calTime = {}

def FloydWarshall(g: networkx.MultiDiGraph):
    start = time.time()
    nodeList = list(g.nodes)
    
    adjArray = {}
    adjArray = {u: {v: (-1, -1) for v in nodeList} for u in nodeList}
    print("Empty adjacent dict in:", time.time() - start)
    start = time.time()
    
    for node in nodeList:
        adjArray[node][node] = (0, node)
        
    edgeList = list(g.edges)
    for u, v, trash in edgeList:
         adjArray[u][v] = (g[u][v][0]['time'], u)
    print("Only intermediate pairs in:", time.time() - start)
    print ("passed")
         
        
    # return
    nodeL = -1
    longestSoFar = -1
    for k in reversed(nodeList):
        print("At", k, end="")
        start = time.time()
        for i in nodeList:
            for j in nodeList:
                if i == j:
                    continue
                if k == i or k == j:
                    continue
                
                if adjArray[i][k][0] == -1 or adjArray[k][j][0] == -1:
                    continue
                
                if adjArray[i][j][0] == -1 or adjArray[i][j][0] > adjArray[i][k][0] + adjArray[k][j][0]:
                    adjArray[i][j] = (adjArray[i][k][0] + adjArray[k][j][0], k)
        
        dur = time.time() - start
        calTime[k] = dur
        if longestSoFar < dur:
            longestSoFar = dur
            nodeL = k
        print(" took:", dur, ", longest so far:", nodeL, "in:", longestSoFar)      
    return adjArray

adj = FloydWarshall(g)


Empty adjacent dict in: 1.634474277496338
Only intermediate pairs in: 0.4219071865081787
passed
At 7686 took: 2.0991671085357666 , longest so far: 7686 in: 2.0991671085357666
At 7685 took: 2.162587881088257 , longest so far: 7685 in: 2.162587881088257
At 7684 took: 2.0559885501861572 , longest so far: 7685 in: 2.162587881088257
At 7683 took: 1.4940078258514404 , longest so far: 7685 in: 2.162587881088257
At 7682 took: 1.5058503150939941 , longest so far: 7685 in: 2.162587881088257
At 7517 took: 1.5824236869812012 , longest so far: 7685 in: 2.162587881088257
At 7518 took: 1.5780808925628662 , longest so far: 7685 in: 2.162587881088257
At 7512 took: 1.4809050559997559 , longest so far: 7685 in: 2.162587881088257
At 7516 took: 1.5352087020874023 , longest so far: 7685 in: 2.162587881088257
At 7511 took: 1.4673316478729248 , longest so far: 7685 in: 2.162587881088257
At 7515 took: 1.5284993648529053 , longest so far: 7685 in: 2.162587881088257
At 7510 took: 1.4754865169525146 , longest so 

In [ ]:
calTime['2252']

NameError: name 'calTime' is not defined

In [ ]:
dic = {}
for i in range(2):
    dic[i] = {}
    dic[i][i + 1] = -1
    
print(dic[1][2])

-1


In [ ]:
edgeL = list(g.edges)
print(len(edgeL))
for u, avoid, i in edgeL:
    if u == 7276 and avoid == 35:
        print(u, avoid)
    # print(g[u][v][0]['time'])
    # break
# print(g)
# g[edgeL[0][0]][edgeL[0][1]][0]

9974


In [ ]:
# Dijkstra Old
def Dijkstra(k: networkx.MultiDiGraph, startingIndex = 0):
    unvisited = list(k.nodes)
    visited = [unvisited.pop(startingIndex)]

    if len(list(networkx.neighbors(k, visited[-1]))) == 0:
        # print("No path to any other node found")
        return
        
    shortestPath = {}
    for node in list(k.nodes):
        if node == visited[0]:
            shortestPath[node] = (node, 0)
        else:
            shortestPath[node] = (-1, -1)
            
    # print(shortestPath)

    while len(unvisited) > 0:
        nodeToConsider = visited[-1]
        
                
        # Update shortestPath
        for node in unvisited:
            dis = shortestPath[node][1]
            
            # Skip node that cant be reached directly
            if networkx.is_path(k, [nodeToConsider, node]) == False:
                continue
            
            weighToHere = shortestPath[nodeToConsider][1]
            minWeight = k[nodeToConsider][node][0]['time']
            if dis == -1:
                shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
            elif shortestPath[node][1] > minWeight + weighToHere:
                    shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
        
        nextNodeToConsider = -1
        minWeightSoFar = -1
        for u in unvisited:
            # Skip node that are not updated yet
            if shortestPath[u][1] == -1:
                continue
            
            # print(shortestPath[u][1])
            if minWeightSoFar == -1:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
            elif shortestPath[u][1] < minWeightSoFar:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
        if nextNodeToConsider == -1:
            # print("Unreachable node found")
            return shortestPath
            
        
        # print(shortestPath)
        # print(f"{nextNodeToConsider=}")
        
        unvisited.remove(nextNodeToConsider)
        visited.append(nextNodeToConsider)
        # break

    # print()
    return shortestPath

Dijkstra(g)

In [ ]:
from rich import print
# print(g.subgraph(list(connected_components(ug))[0]))
grph: networkx.MultiDiGraph = g.subgraph(list(connected_components(ug))[0]).copy()
stops = list(grph.nodes)
stops.sort()
print(len(stops))